In [1]:
import numpy as np
from glob import glob
import cv2                
import matplotlib.pyplot as plt   
from tqdm import tqdm_notebook as tqdm
import torch
import torchvision.models as models
from PIL import Image
import torch.nn as nn
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import torch.optim as optim
import torch.nn.functional as F
import os
from torchvision import datasets
import torchvision.transforms as transforms
%matplotlib inline
import split_folders
ImageFile.LOAD_TRUNCATED_IMAGES = True
# check if CUDA is available
use_cuda = torch.cuda.is_available()

In [2]:
# Directory paths to all data (train, valid and test)
data_dir = r'D:\All_Data\Art_Project\Extracted_Data'

# split_folders.ratio(data_dir, output=data_dir + "\\output", seed=1337, ratio=(.8, .1, .1))

In [3]:
train_dir = os.path.join(data_dir, 'output/train/')
valid_dir = os.path.join(data_dir, 'output/val/')
test_dir = os.path.join(data_dir, 'output/test/')

# All pre-trained models expect input images normalized in the same way, 
# i.e. mini-batches of 3-channel RGB images of shape (3 x H x W), 
# where H and W are expected to be at least 224. 
# The images have to be loaded in to a range of [0, 1] and 
# then normalized using mean = [0.485, 0.456, 0.406] and std = [0.229, 0.224, 0.225]
# Referenced from: https://pytorch.org/docs/stable/torchvision/models.html
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(), # randomly flip and rotate
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], 
                         [0.229, 0.224, 0.225])
    ])

# We do similar tranformations to validation except Random flips and rotations
# After going through the first review and only resizing the image to 224, I was getting following error:
# Unknown resampling filter (224)
# Hence first, Resizing to a higher value and then CenterCrop the right size.
valid_test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], 
                         [0.229, 0.224, 0.225])
    ])

In [4]:
# The following import is required for training to be robust to truncated images

# Keep a track of losses, for plotting
losses = {'train':[], 'validation':[]}

# Train function has been referenced from the function we used in cifar exercise.
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in tqdm(range(1, n_epochs+1)):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(tqdm(loaders['train'])):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## find the loss and update the model parameters accordingly
            ## record the average training loss, using something like
            ## train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
            optimizer.zero_grad()
            
            pred = model(data)
            
            loss = criterion(pred, target)
            
            train_loss += ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
            loss.backward()
            optimizer.step()            
            
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(tqdm(loaders['valid'])):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## update the average validation loss
            val_pred = model(data)
            val_loss = criterion(val_pred, target)
            
            valid_loss += ((1 / (batch_idx + 1)) * (val_loss.data - valid_loss))
            
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        ## TODO: save the model if validation loss has decreased
        if (valid_loss < valid_loss_min):
            print("Saving model.  Validation loss:... {} --> {}".format(valid_loss_min, valid_loss.item()))
            valid_loss_min = valid_loss
            torch.save(model.state_dict(), save_path)
            print()
            
        losses['train'].append(train_loss)
        losses['validation'].append(valid_loss)
    # return trained model
    return model

In [ ]:
# Test function is also referenced from cifar exercise.
# After multiple iterations, the architecture of model was selected, making sure the accuracy is > 10%

def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(tqdm(loaders['test'])):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

In [ ]:
model_transfer = models.vgg19_bn(pretrained=True)
for param in model_transfer.features.parameters():
    param.requires_grad = False

n_inputs = model_transfer.classifier[6].in_features
last_layer = nn.Linear(n_inputs, 27)
model_transfer.classifier[6] = last_layer



# For Resnet50
# Referenced from https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
# model_transfer = models.resnet50(pretrained=True)
# num_ftrs = model_transfer.fc.in_features
# model_transfer.fc = nn.Linear(num_ftrs, 27)

In [ ]:
if use_cuda:
    model_transfer = model_transfer.cuda()
    
print(model_transfer)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256, kernel_size=(3, 3)

In [ ]:
criterion_transfer = nn.CrossEntropyLoss()
optimizer_transfer = optim.Adam(model_transfer.parameters(), lr=0.00035)

In [ ]:
train_data = datasets.ImageFolder(train_dir, transform=train_transform)
valid_data = datasets.ImageFolder(valid_dir, transform=valid_test_transform)
test_data = datasets.ImageFolder(test_dir, transform=valid_test_transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(valid_data, batch_size=32)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32)

print('Number of training images: {}'.format(len(train_data)))
print('Number of validation images: {}'.format(len(valid_data)))
print('Number of testing images: {}'.format(len(test_data)))

Number of training images: 65146
Number of validation images: 8133
Number of testing images: 8167


In [ ]:
## TODO: Specify data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=True)
val_loader = torch.utils.data.DataLoader(valid_data, batch_size=16)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=16)

loaders_transfer = {'train': train_loader, 'valid': val_loader, 'test': test_loader}

In [ ]:
model_transfer = train(10, loaders_transfer, model_transfer, optimizer_transfer, criterion_transfer, use_cuda, r'D:\All_Data\Art_Project\Model\model_transfer_all_data.pt')

C:\Users\aksha\AppData\Local\conda\conda\envs\image_tagging\lib\site-packages\PIL\Image.py:2731: DecompressionBombWarning: Image size (107327830 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,
C:\Users\aksha\AppData\Local\conda\conda\envs\image_tagging\lib\site-packages\PIL\Image.py:2731: DecompressionBombWarning: Image size (99962094 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,


Epoch: 1 	Training Loss: 2.271307 	Validation Loss: 1.925872
Saving model.  Validation loss:... inf --> 1.9258719682693481



Epoch: 2 	Training Loss: 2.149526 	Validation Loss: 1.920969
Saving model.  Validation loss:... 1.9258719682693481 --> 1.9209693670272827



Epoch: 3 	Training Loss: 2.135283 	Validation Loss: 1.841356
Saving model.  Validation loss:... 1.9209693670272827 --> 1.8413563966751099



Epoch: 4 	Training Loss: 2.116113 	Validation Loss: 1.876220


Epoch: 5 	Training Loss: 2.109187 	Validation Loss: 1.827786
Saving model.  Validation loss:... 1.8413563966751099 --> 1.827785611152649



Epoch: 6 	Training Loss: 2.101492 	Validation Loss: 1.815805
Saving model.  Validation loss:... 1.827785611152649 --> 1.815805196762085



Epoch: 7 	Training Loss: 2.079574 	Validation Loss: 1.779942
Saving model.  Validation loss:... 1.815805196762085 --> 1.7799421548843384



Epoch: 8 	Training Loss: 2.077747 	Validation Loss: 1.727409
Saving model.  Validation loss:... 1.7799421548843384 --> 1.7274086475372314



Epoch: 9 	Training Loss: 2.070432 	Validation Loss: 1.709242
Saving model.  Validation loss:... 1.7274086475372314 --> 1.7092418670654297



In [ ]:
# load the model that got the best validation accuracy (uncomment the line below)
model_transfer.load_state_dict(torch.load(r'D:\All_Data\Art_Project\Model\model_transfer_all_data.pt'))

In [ ]:
# Test function is also referenced from cifar exercise.
# After multiple iterations, the architecture of model was selected, making sure the accuracy is > 10%

def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(tqdm(loaders['test'])):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

In [ ]:
test(loaders_transfer, model_transfer, criterion_transfer, use_cuda)